## 📏 Diferencias_entre_duplicados_en_pb.ipynb

**Autor:** Johanna Atenea Carreon Baltazar  
**Contacto:** johannaatenea13@gmail.com
**Fecha de última modificación:** 23 de junio de 2025

---

### 🎯 Objetivo

Calcular las **distancias físicas (en kilobases)** entre genes duplicados en cada genoma del grupo de estudio.  
Estas distancias permiten explorar patrones de organización genómica asociados a duplicaciones, considerando tanto la separación lineal entre genes consecutivos como la distancia circular en el genoma.

---

### 📥 Entradas requeridas

- `genes_secundarios_k_means.pkl` y `genes_secundarios_umbrales.pkl`: diccionarios con genes duplicados por genoma (clasificados como secundarios).
- `longitudes_copias.csv`: longitudes genómicas (en pb y kb) para cada genoma.
- Archivos de anotaciones por genoma (`*.txt`) que contienen información detallada sobre cada gen.
- Script bash: `script_obtener_posiciones.sh`: extrae posiciones genómicas de los genes duplicados.
- Archivos resultantes del script bash:
  - `posiciones_k_means.csv`
  - `posiciones_umbrales.csv`

---

### 📤 Salidas generadas

- Diccionarios:
  - `resultados_diferencias_genes_k_means`
  - `resultados_diferencias_genes_umbrales`
  Estos contienen las distancias (en kilobases) entre genes duplicados en cada genoma.
  
- Estadísticas descriptivas para evaluar la dispersión, agrupamiento o separación de los genes duplicados en el espacio genómico.

---

### 🧰 Librerías requeridas

```python
import pickle
import csv
import pandas as pd
import numpy as np


# Pipeline
---


### 1. Importación de los archivos `genes_secundarios_k_means.pkl` y `genes_secundarios_umbrales.pkl`.

Este archivo contiene un diccionario donde las llaves son los identificadores de los genomas y los valores son listas de identificadores de genes duplicados en cada genoma.

**Ejemplo de estructura del diccionario:**

```python
genes_secundarios_k_means = {
    2914041.10: [4092, 4899, 903, 1259, 3694, 2151, 936, 4764, 5875, 2666, 1072, 1013, 655, 1058, 5341, 977],
    ...
```
---
### 2. Importamos las longitudes de los genomas 
El archivo longitudes_copias.csv contiene las longitudes de cada genoma presente en el diccionario genes_copia. Las longitudes están medidas en kilobases (Kb) y pares de bases (pb).
![](longitudes.png)

---

### 3.Archivos de anotaciones para cada genoma
Para cada genoma que contenga genes duplicados tenemos un archivo cuyo nombre tiene la estructura *id_genoma.txt*. Este archivo contiene varias columnas, de las cuales las más relevantes para nosotros son :
- **contig_id**: 
   - Identificador del *contig* en el que se encuentra el gen. Este campo indica en qué parte del genoma ensamblado está localizado el gen.

2. **feature_id**:
   - Identificador único de la característica o elemento genómico (en este caso, el gen) en el sistema de referencia, aquí en el formato `fig|6666666.212069.peg.1`.

3. **type**:
   - Tipo de característica genética. Aquí parece ser `peg`, que comúnmente se usa para representar. 
    
4. **location**:
   - Ubicación completa del gen en el ensamblaje, generalmente en el formato `contig_id_start_stop`. Por ejemplo, `gi|428159384|gb|AJLM01000188.1|_151_1137` indica que el gen se encuentra en el contig `AJLM01000188.1` y abarca desde la posición `151` hasta `1137`.

5. **start**:
   - La posición inicial(primer nucleótido) del gen dentro del *contig* en el que se encuentra secuencia genómica.

6. **stop**:
   - La posición final(último nucleotido) del gen dentro del *contig*.

7. **strand**:
   - La cadena o hebra (`+` o `-`) en la que se encuentra el gen. El valor `+` significa que el gen está en la hebra principal, y `-` indica la hebra complementaria.
   - 
**Ejemplo de estructura de la tabla:**
![](2911041.10.png)

---
### 4. Obtenciones de las posiciones en KB para cada gen duplicado de cada genoma
Utilizamos los diccionarios `genes_secundarios_k_means`,`genes_secundarios_umbrales` y los archivos .txt mencionados en el punto 3 para obtener las posiciones en kilobases de cada gen duplicado. Para ello, empleamos un script en Bash llamado script_obtener_posiciones.sh. Este script realiza lo siguiente:

- Toma un genoma que sea llave en el diccionario  `genes_secundarios_k_means`\ `genes_secundarios_umbrales`.

-  Busca el archivo correspondiente al genoma, es decir, id_genoma.txt.

- Extrae los valores de las columnas genome_id, feature_id, contig_id, start y stop para cada gen duplicado y los guarda en archivos llamados `posiciones_k_means.csv`\ `posiciones_umbrales.csv` respectivamente.

- Repite el proceso para cada genoma en el diccionario

---
### 5. Calcular las diferencias entre los genes duplicados de cada genoma en Kilobases.
 Las diferencias entre los genes duplicados en cada genoma se almacenan en un diccionario llamado `resultados_diferencias_genes_k_means`\ `resultados_diferencias_genes_umbrales` según sea el caso, cuya estructura es la siguiente:
```
resultados_diferencias_genes_k_means = {
    id_genoma: [diferencias en número de genes entre duplicados],
    ...
}
```

Tales diferencias se calculan con la función calcular_diferencias_genes, la cual en este caso mide las distancias en términos del número de genes que separan a cada par de genes duplicados dentro del mismo genoma. Esto se hace en dos pasos principales:

- Diferencias en número de genes entre duplicados consecutivos

Para cada par de genes duplicados consecutivos dentro del genoma, se cuenta cuántos genes los separan en la lista ordenada por posición de inicio (start):
$$ \text{distancia} = \frac{\max\{p_{1,\text{stop}}, p_{1,\text{start}}\} - \min\{p_{2,\text{stop}}, p_{2,\text{start}}\}}{1000} $$

Donde $p_1$ y $p_2$ son los genes duplicados consecutivos en el genoma.

- Diferencia circular entre el último y el primer gen duplicado
    $$ \text{distancia\_circular} = \frac{\text{longitud del genoma} - \max\{p_{\text{last},\text{stop}}, p_{\text{last},\text{start}}\} + \min\{p_{\text{first},\text{stop}}, p_{\text{first},\text{start}}\}}{1000} $$
Se considera también la distancia circular entre el último gen duplicado y el primero, es decir, cuántos genes hay entre ellos al recorrer el genoma de manera circular. Donde la  longitud del genoma es la longitud del genoma en  pares de bases.
---
5. Estadisticas descriptivas de los diccionarios que almacena estas diferencias.

## Importar los diccionarios que contienen los genes duplicados

In [6]:
import pickle
import csv
import pandas as pd
import numpy as np

### Importar los datos obtenidos con K-means

In [29]:
# Cargar el archivo .pkl
with open('genes_secundarios_k_means.pkl', 'rb') as file:
    genes_secundarios_k_means= pickle.load(file)

# Verificar el contenido
#print(type(genes_copia))  # Para saber qué tipo de objeto es
print(genes_secundarios_k_means)  # Muestra una parte del contenido

{2490939.1: [317, 321, 322, 334, 503, 608, 993, 1308, 1494, 1565, 2005, 2008, 2193, 2199, 2660, 2792, 2955, 3024, 3350, 3382, 3596, 3719, 3792, 3793, 3970, 4309, 4507, 4588, 4614, 4940, 5162, 5175, 5508, 5720, 5722, 5959, 6197, 6231], 103690.82: [665, 693, 1148, 1286, 1512, 1631, 1655, 1672, 1809, 2064, 2105, 2315, 2460, 2688, 2936, 2956, 3081, 3286, 3468, 3469, 4855, 4867, 4870, 5161, 5392, 5515, 5803, 5805, 5835], 211165.2: [198, 276, 365, 438, 653, 667, 1120, 1234, 1335, 1337, 1568, 1762, 1803, 2208, 2238, 2239, 2290, 2354, 2380, 2812, 2935, 3193, 3227, 3340, 3471, 3473, 3687, 3793, 3806, 3824, 3954, 3972, 4147, 4203, 4317, 4367, 4396, 4489, 4557, 4701, 4802, 4805, 4862, 5039, 5204, 5506, 6213, 6355, 6359, 6372, 6425, 6443, 6444, 6483, 6608, 6648, 6854, 6868, 6969, 7210], 1472755.9: [88, 430, 868, 915, 921, 1182, 1346, 1623, 1739, 2005, 2257, 3031, 3250, 3334, 3561, 4003, 4343, 4345, 4763, 5327, 5612, 5747, 6272, 6674, 6826, 7090, 7114, 7122, 7160, 7307], 2576904.6: [14, 833, 899, 1

### Importar los datos obtenidos usando clasificación con umbrales

In [11]:
# Cargar el archivo .pkl
with open('genes_secundarios_umbrales.pkl', 'rb') as file:
    genes_secundarios_umbrales = pickle.load(file)

# Verificar el contenido
#print(type(genes_copia))  # Para saber qué tipo de objeto es
print(genes_secundarios_umbrales)  # Muestra una parte del contenido

{2490939.1: [248, 317, 321, 322, 334, 378, 503, 525, 608, 895, 993, 1106, 1234, 1494, 1565, 1597, 2005, 2144, 2186, 2194, 2199, 2660, 2764, 2882, 2955, 3024, 3350, 3382, 3461, 3655, 3700, 3719, 3792, 3793, 3796, 3970, 4309, 4507, 4588, 4614, 4940, 5063, 5150, 5162, 5173, 5175, 5381, 5518, 5578, 5720, 5722, 5959, 6151, 6197, 6231, 6283, 6284, 6349], 103690.82: [98, 189, 190, 378, 469, 665, 693, 1148, 1286, 1512, 1631, 1655, 1672, 1809, 2064, 2086, 2105, 2315, 2460, 2688, 2766, 2936, 2956, 3002, 3081, 3195, 3286, 3468, 3469, 3472, 3552, 3998, 4086, 4564, 4646, 4855, 4867, 4870, 5161, 5220, 5392, 5435, 5515, 5579, 5611, 5619, 5791, 5835], 211165.2: [147, 206, 276, 312, 365, 438, 509, 655, 667, 794, 1120, 1172, 1234, 1335, 1337, 1762, 1803, 2042, 2239, 2380, 2410, 2513, 2812, 2935, 3193, 3227, 3280, 3303, 3340, 3471, 3473, 3793, 3805, 3824, 3954, 3955, 3972, 4147, 4317, 4396, 4477, 4489, 4557, 4701, 4802, 4862, 4925, 5039, 5204, 5279, 5338, 5506, 5566, 6027, 6083, 6213, 6277, 6355, 6359, 6

## Longitud de los genomas

In [3]:
# Cargar el archivo CSV como DataFrame
longitudes_copias = pd.read_csv("longitudes_copias.csv", delimiter='\t', header=None, names=['id', 'Longitud(bp)'])
longitudes_copias['Longitud(KB)'] = longitudes_copias['Longitud(bp)'] / 1000
# Mostrar las primeras 5 filas del DataFrame
longitudes_copias

,id,Longitud(bp),Longitud(KB)
0,103690.82,6413771,6413.771
1,1472755.90,7733505,7733.505
2,1618022.90,7061466,7061.466
3,1647413.14,5705437,5705.437
4,1751286.15,6463965,6463.965
5,1869241.20,6990729,6990.729
6,1914872.23,5294286,5294.286
7,2038116.21,8363872,8363.872
8,211165.20,7409323,7409.323
9,2490939.10,7013200,7013.200


### Transformar los diccionarios al formato necesario para ejecutar el script `script_obtener_posiciones.sh`

#### k-means

In [ ]:

# Cargar el diccionario desde un archivo .pkl
with open("genes_secundarios_k_means.pkl", "rb") as pklfile:
    genes_secundarios_k_means = pickle.load(pklfile)

# Guardar el resultado en un archivo CSV
with open("genes_secundarios_k_means.csv", "w", newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    for key, values in genes_secundarios_k_means.items():
        values_str = ", ".join(map(str, values))
        writer.writerow([key, values_str])
        # Imprimir en pantalla
        print(f"{key}\t{values_str}")

#### umbrales

In [ ]:
# Crear un nuevo diccionario para almacenar las estadísticas
estadisticas = {}

# Calcular estadísticas para cada genoma
for id_genoma, diferencias in resultados_diferencias_corregido.items():
    estadisticas[id_genoma] = {
        'media (KB)': np.mean(diferencias),
        'mediana (KB)': np.median(diferencias),
        'desviación estándar (KB)': np.std(diferencias),
        'máximo (KB)': np.max(diferencias),
        'mínimo (KB)': np.min(diferencias)
    }

# Mostrar las estadísticas
for id_genoma, stats in estadisticas.items():
    print(f'Estadísticas para Genoma {id_genoma}:')
    for key, value in stats.items():
        print(f'  {key}: {value:.2f}')
    print()

In [17]:
# Diccionario con los valores proporcionados
datos = {
    "2490939.1": 38,
    "103690.82": 29,
    "211165.2": 60,
    "1472755.9": 30,
    "2576904.6": 31,
    "2764711.14": 43,
    "3134896.7": 52,
    "3349875.4": 33,
    "63737.69": 40,
    "272123.44": 28,
    "1618022.9": 26,
    "2572090.7": 31,
    "2576902.6": 27,
    "3025190.14": 26,
    "1914872.23": 23,
    "1647413.14": 25,
    "2038116.21": 33,
    "2653204.7": 25,
    "28072.26": 44,
    "76335.23": 32,
    "446679.11": 23,
    "449208.14": 24,
    "1869241.2": 34,
    "2576903.5": 34,
    "3349876.5": 32,
    "2914041.1": 31
}

# Sumar todos los valores de la segunda columna
suma_total = sum(datos.values())

# Imprimir el resultado
print(f"La suma total es: {suma_total}")


La suma total es: 854


In [12]:
# esta función ayuda a pasar el diccionario genes_copia al formato que ocupo para despues correr el script en shell que 
# me ayudara a obtener las distancias
# Cargar el diccionario desde un archivo .pkl
with open("genes_secundarios_umbrales.pkl", "rb") as pklfile:
    genes_secundarios_umbrales= pickle.load(pklfile)

# Guardar el resultado en un archivo CSV
with open("genes_secundarios_umbrales.csv", "w", newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    for key, values in genes_secundarios_k_means.items():
        values_str = ", ".join(map(str, values))
        writer.writerow([key, values_str])
        # Imprimir en pantalla
        print(f"{key}\t{values_str}")

2490939.1	248, 317, 321, 322, 334, 378, 503, 525, 608, 895, 993, 1106, 1234, 1494, 1565, 1597, 2005, 2144, 2186, 2194, 2199, 2660, 2764, 2882, 2955, 3024, 3350, 3382, 3461, 3655, 3700, 3719, 3792, 3793, 3796, 3970, 4309, 4507, 4588, 4614, 4940, 5063, 5150, 5162, 5173, 5175, 5381, 5518, 5578, 5720, 5722, 5959, 6151, 6197, 6231, 6283, 6284, 6349
103690.82	98, 189, 190, 378, 469, 665, 693, 1148, 1286, 1512, 1631, 1655, 1672, 1809, 2064, 2086, 2105, 2315, 2460, 2688, 2766, 2936, 2956, 3002, 3081, 3195, 3286, 3468, 3469, 3472, 3552, 3998, 4086, 4564, 4646, 4855, 4867, 4870, 5161, 5220, 5392, 5435, 5515, 5579, 5611, 5619, 5791, 5835
211165.2	147, 206, 276, 312, 365, 438, 509, 655, 667, 794, 1120, 1172, 1234, 1335, 1337, 1762, 1803, 2042, 2239, 2380, 2410, 2513, 2812, 2935, 3193, 3227, 3280, 3303, 3340, 3471, 3473, 3793, 3805, 3824, 3954, 3955, 3972, 4147, 4317, 4396, 4477, 4489, 4557, 4701, 4802, 4862, 4925, 5039, 5204, 5279, 5338, 5506, 5566, 6027, 6083, 6213, 6277, 6355, 6359, 6372, 6483, 

In [25]:
# Ejemplo de n arrchivo  txt con el que vamos  a trabajar
df = pd.read_csv('Cianobacterias_txt/2914041.10.txt', sep='\t')
df[150:155]

,contig_id,feature_id,type,location,start,stop,strand,function,aliases,figfam,evidence_codes,nucleotide_sequence,aa_sequence
150,NZ_CP091913.1,fig|2914041.10.peg.136,peg,NZ_CP091913.1_149435_150781,149435.0,150781.0,+,Serine/threonine kinase,NaN,NaN,NaN,atgaaaatatgtcctaatattggctgtttatatccccaaaaccccg...,MKICPNIGCLYPQNPDTATVCLKCGEQLLLNHRYHLLRQIGHGGFG...
151,NZ_CP091913.1,fig|2914041.10.repeat.14,repeat,NZ_CP091913.1_150852_150959,150852.0,150959.0,+,repeat region,NaN,NaN,NaN,cgccccgctccgctaacgcaattaagtgttgtaacggggatttaga...,NaN
152,NZ_CP091913.1,fig|2914041.10.peg.137,peg,NZ_CP091913.1_151044_151829,151044.0,151829.0,+,Sirohydrochlorin cobaltochelatase CbiX(long) (...,NaN,NaN,NaN,atgccctctgcttatttgctagtatctcacggaagccgcgatcgcc...,MPSAYLLVSHGSRDRRPEIAMQQLAVLVANKLQIPEKLVGTASLEV...
153,NZ_CP091913.1,fig|2914041.10.peg.138,peg,NZ_CP091913.1_152120_152899,152120.0,152899.0,+,Uroporphyrinogen-III methyltransferase (EC 2.1...,NaN,NaN,idu(1);Heme_and_Siroheme_Biosynthesis idu(1);C...,atgaaccgcacagacaaggaattacaaaagagtttgggtaaagttt...,MNRTDKELQKSLGKVYLVGAGPGDPGLMTLKGKSILECADVVIYDA...
154,NZ_CP091913.1,fig|2914041.10.peg.139,peg,NZ_CP091913.1_153071_153784,153071.0,153784.0,+,hypothetical protein,NaN,NaN,NaN,atgttgaatagcacaaacccagcagatactgaatttgggcggctct...,MLNSTNPADTEFGRLLWEIVRRPKLPQCPLPGSKISPQNNTALVCQ...


### Ejecución del script `script_obtener_posiciones.sh`e importar los archivos resultantes `posiciones_umbrales.csv`y `posiciones_k_means.csv`respectivamente.

se creo un script en la terminal para extrae las posiciones de los genes duplicados, este archivo se llama posiciones.csv y contiene exactamente las columnas genoma,contig_id,gen,start, stop las dos ultimos corresponden a las posiciones donde empieza y termina
Este  archivo se encuentra en la carpeta /files/atenea/bacterias/Cianobacterias_corregidas/posiciones de user6@geomtop.matmor.unam.mx
con el nombre **script_obtener_posiciones.sh**. Al correr este script obtengo como resultado el archivo *posiciones.csv** en el cual se tiene identificado los genes copia de cada genoma, en que respectivo con tig(al ser genomas completos todos pertencen al mismo) y en que nucleotido y empiezan y terminan.

### k-means

In [34]:
# Cargar el archivo CSV como un DataFrame
posiciones_copias_k_means = pd.read_csv('posiciones_k_means.csv',sep ='\t')
posiciones_copias_k_means

,genome_id,gen_id,contig_id,start,stop
0,103690.82,665,NC_003272.1,708712,710685
1,103690.82,693,NC_003272.1,741096,738640
2,103690.82,1148,NC_003272.1,1236087,1233670
3,103690.82,1286,NC_003272.1,1388613,1385845
4,103690.82,1512,NC_003272.1,1631448,1633787
...,...,...,...,...,...
849,76335.23,5990,NZ_CP046703.1,6398941,6400629
850,76335.23,6400,NZ_CP046703.1,6830215,6832080
851,76335.23,6649,NZ_CP046703.1,7105113,7103431
852,76335.23,6692,NZ_CP046703.1,7162000,7163661


### umbrales

In [35]:
# Cargar el archivo CSV como un DataFrame
posiciones_copias_umbrales = pd.read_csv('posiciones_umbrales.csv',sep ='\t')
posiciones_copias_umbrales

,genome_id,gen_id,contig_id,start,stop
0,103690.82,98,NC_003272.1,95168,97051
1,103690.82,189,NC_003272.1,188377,186584
2,103690.82,190,NC_003272.1,190316,188577
3,103690.82,378,NC_003272.1,398960,400573
4,103690.82,469,NC_003272.1,512093,509313
...,...,...,...,...,...
1227,76335.23,6649,NZ_CP046703.1,7105113,7103431
1228,76335.23,6656,NZ_CP046703.1,7112268,7113893
1229,76335.23,6692,NZ_CP046703.1,7162000,7163661
1230,76335.23,6701,NZ_CP046703.1,7172726,7174243


### Función para filtrar las posiciones de los genes duplicados para un genoma específico

### k-means

In [40]:
# Función para calcular las diferencias entre posiciones de genes duplicados en el mismo genoma ensamblado
def procesar_posiciones(df_posiciones, longitud_genoma):
    # Ordenar las posiciones por el campo 'start'
    posiciones_ordenadas = df_posiciones.sort_values(by='start')
    
    diferencias = []
    for i in range(1, len(posiciones_ordenadas)):
        p1 = posiciones_ordenadas.iloc[i - 1]  # Gen anterior
        p2 = posiciones_ordenadas.iloc[i]      # Gen actual
        
        # Calcular la distancia física entre p1 y p2
        distancia = max(p1['start'], p1['stop']) - min(p2['start'], p2['stop'])
        diferencias.append(abs(distancia) / 1000)  # Convertir a kilobases (KB)
    
    # Considerar la distancia circular (del último gen al primero)
    if len(posiciones_ordenadas) > 1:
        p1 = posiciones_ordenadas.iloc[-1]  # Último gen
        p2 = posiciones_ordenadas.iloc[0]   # Primer gen
        diferencia_circular = (longitud_genoma - max(p1['start'], p1['stop'])) + min(p2['start'], p2['stop'])
        diferencias.append(diferencia_circular / 1000)
    
    return diferencias

# Función para calcular las diferencias entre genes duplicados para un genoma específico
def calcular_diferencias(df_posiciones, df_longitudes, id_genoma):
    # Filtrar las posiciones de genes duplicados para el genoma
    posiciones_duplicados = df_posiciones[df_posiciones['genome_id'] == id_genoma]
    
    # Obtener la longitud del genoma
    longitud_genoma = df_longitudes.loc[df_longitudes['id'] == id_genoma, 'Longitud(bp)'].values[0]
    
    # Calcular las diferencias entre genes duplicados
    diferencias = procesar_posiciones(posiciones_duplicados, longitud_genoma)
    
    return diferencias

# Calcular las diferencias entre genes duplicados para cada genoma
resultados_diferencias_k_means = {}

for id_genoma in posiciones_copias_k_means['genome_id'].unique():
    resultados_diferencias_k_means[id_genoma] = calcular_diferencias(posiciones_copias_k_means, longitudes_copias, id_genoma)

# Mostrar los resultados
print(resultados_diferencias_k_means)

{103690.82: [27.955, 492.574, 149.758, 242.835, 112.269, 18.475, 13.226, 159.211, 292.866, 49.606, 230.723, 150.844, 264.411, 301.786, 34.597, 130.045, 216.455, 180.321, 0.024, 1492.116, 14.825, 0.701, 296.858, 260.885, 128.426, 330.112, 0.619, 32.343, 729.804], 1472755.9: [359.549, 463.285, 45.696, 2.701, 243.312, 191.922, 268.811, 135.853, 272.953, 318.956, 769.342, 221.987, 83.298, 225.854, 452.11, 345.746, 0.397, 381.058, 589.569, 278.154, 145.539, 535.972, 400.784, 211.093, 331.648, 20.738, 9.871, 35.422, 165.268, 168.081], 1618022.9: [593.324, 372.309, 291.381, 539.154, 6.127, 0.791, 260.413, 70.723, 133.842, 992.659, 23.359, 7.17, 294.066, 338.139, 526.39, 543.869, 253.545, 0.511, 223.998, 532.004, 59.144, 40.601, 92.732, 52.284, 308.467, 455.155], 1647413.14: [26.995, 19.623, 132.43, 21.7, 225.579, 89.507, 210.768, 39.133, 227.249, 84.327, 0.079, 1022.45, 222.277, 502.796, 155.831, 110.864, 30.913, 52.634, 0.053, 294.353, 793.698, 583.782, 360.506, 0.161, 449.316], 1869241.2: [

### umbrales

In [43]:
# Función para calcular las diferencias entre posiciones de genes duplicados en el mismo genoma ensamblado
def procesar_posiciones(df_posiciones, longitud_genoma):
    # Ordenar las posiciones por el campo 'start'
    posiciones_ordenadas = df_posiciones.sort_values(by='start')
    
    diferencias = []
    for i in range(1, len(posiciones_ordenadas)):
        p1 = posiciones_ordenadas.iloc[i - 1]  # Gen anterior
        p2 = posiciones_ordenadas.iloc[i]      # Gen actual
        
        # Calcular la distancia física entre p1 y p2
        distancia = max(p1['start'], p1['stop']) - min(p2['start'], p2['stop'])
        diferencias.append(abs(distancia) / 1000)  # Convertir a kilobases (KB)
    
    # Considerar la distancia circular (del último gen al primero)
    if len(posiciones_ordenadas) > 1:
        p1 = posiciones_ordenadas.iloc[-1]  # Último gen
        p2 = posiciones_ordenadas.iloc[0]   # Primer gen
        diferencia_circular = (longitud_genoma - max(p1['start'], p1['stop'])) + min(p2['start'], p2['stop'])
        diferencias.append(diferencia_circular / 1000)
    
    return diferencias

# Función para calcular las diferencias entre genes duplicados para un genoma específico
def calcular_diferencias(df_posiciones, df_longitudes, id_genoma):
    # Filtrar las posiciones de genes duplicados para el genoma
    posiciones_duplicados = df_posiciones[df_posiciones['genome_id'] == id_genoma]
    
    # Obtener la longitud del genoma
    longitud_genoma = df_longitudes.loc[df_longitudes['id'] == id_genoma, 'Longitud(bp)'].values[0]
    
    # Calcular las diferencias entre genes duplicados
    diferencias = procesar_posiciones(posiciones_duplicados, longitud_genoma)
    
    return diferencias

# Calcular las diferencias entre genes duplicados para cada genoma
resultados_diferencias_umbrales = {}

for id_genoma in posiciones_copias_umbrales['genome_id'].unique():
    resultados_diferencias_umbrales[id_genoma] = calcular_diferencias(posiciones_copias_umbrales, longitudes_copias, id_genoma)
# Mostrar los resultados
print(resultados_diferencias_umbrales)

{103690.82: [89.533, 0.2, 208.644, 108.74, 196.619, 27.955, 492.574, 149.758, 242.835, 112.269, 18.475, 13.226, 159.211, 292.866, 27.119, 20.781, 230.723, 150.844, 264.411, 79.48, 220.627, 34.597, 45.32, 83.058, 131.832, 82.416, 180.321, 0.024, 2.508, 81.041, 486.333, 98.59, 504.895, 77.727, 227.405, 14.825, 0.701, 296.858, 58.118, 200.797, 46.068, 80.79, 87.291, 27.692, 10.869, 183.562, 49.153, 116.26], 1472755.9: [335.328, 97.888, 388.224, 45.696, 2.701, 243.312, 6.6, 183.094, 25.736, 110.02, 127.24, 71.173, 28.039, 33.313, 138.782, 0.166, 69.417, 59.041, 315.897, 117.872, 19.129, 486.946, 34.609, 80.359, 333.404, 225.854, 452.11, 345.746, 383.371, 83.416, 253.101, 13.63, 233.428, 73.801, 202.698, 145.539, 332.447, 201.309, 163.825, 77.253, 156.381, 7.296, 202.196, 331.648, 20.738, 9.871, 35.422, 165.268, 13.13, 152.636], 1618022.9: [217.387, 173.778, 107.559, 69.566, 22.752, 276.738, 291.381, 238.165, 296.308, 0.987, 6.127, 0.791, 61.998, 12.595, 182.354, 70.723, 889.946, 233.131, 2

### Ejemplo de como funciona el código

In [44]:
import pandas as pd
# Crear el DataFrame con las posiciones de genes duplicados
posiciones_copias1 = pd.DataFrame({
        "genome_id": 1914872.23,
        "gen_id": [846, 1148, 1149, 1201, 1343, 1355, 1493, 2149, 
                   2319, 2556, 3407, 4262, 4367, 4392, 4642, 5014],
        "contig_id": ["NZ_CP020114.1"] * 16,  # Se omite en la función
        "start": [914806, 1257459, 1259897, 1317175, 1469775, 1478624,
                  1632255, 2252172, 2428544, 2664667, 3558071, 4449299, 
                  4548684, 4580937, 4830175, 5215133],
        "stop": [913391, 1258517, 1258608, 1316213, 1468492, 1477422,
                 1633457, 2253263, 2429929, 2666076, 3557181, 4450435, 
                 4549925, 4579978, 4828871, 5216221]
    })

# Crear el DataFrame con las longitudes de los genomas
longitudes_copias = pd.DataFrame({
        "id": [1914872.23],
        "Longitud(bp)": [5294286],
        "Longitud(KB)": [5294.286]})


# Ejecutar la función y mostrar el resultado
genome_id = posiciones_copias1["genome_id"].iloc[0]  # Obtener el ID del genoma
start_positions = posiciones_copias1["start"].tolist()  # Lista de posiciones de inicio
stop_positions = posiciones_copias1["stop"].tolist()  # Lista de posiciones de fin
genome_length = longitudes_copias[longitudes_copias["id"] == genome_id]["Longitud(KB)"].item()  # Longitud total del genoma
diferencias_resultado = calcular_diferencias(posiciones_copias1, longitudes_copias, 1914872.23)
# Imprimir los resultados
print(f"Genome ID: {genome_id}")
print(f"Start positions: {start_positions}")
print(f"Stop positions: {stop_positions}")
print(f"Genome length (bp): {genome_length}")
print(f"Differences array: {diferencias_resultado}")

Genome ID: 1914872.23
Start positions: [914806, 1257459, 1259897, 1317175, 1469775, 1478624, 1632255, 2252172, 2428544, 2664667, 3558071, 4449299, 4548684, 4580937, 4830175, 5215133]
Stop positions: [913391, 1258517, 1258608, 1316213, 1468492, 1477422, 1633457, 2253263, 2429929, 2666076, 3557181, 4450435, 4549925, 4579978, 4828871, 5216221]
Genome length (bp): 5294.286
Differences array: [342.653, 0.091, 56.316, 151.317, 7.647, 153.631, 618.715, 175.281, 234.738, 891.105, 891.228, 98.249, 30.053, 247.934, 384.958, 991.456]


### Estadisticas para cada uno de mis 29 genomas

### k-means

In [59]:
# Crear un nuevo diccionario para almacenar las estadísticas
estadisticas_k_means = {}

# Calcular estadísticas para cada genoma
for id_genoma, diferencias in resultados_diferencias_k_means.items():
    diferencias = np.array(diferencias)  # Convertir a un array de NumPy para evitar errores
    estadisticas_k_means[id_genoma] = {
        'media (KB)': np.mean(diferencias) if diferencias.size > 0 else np.nan,
        'mediana (KB)': np.median(diferencias) if diferencias.size > 0 else np.nan,
        'desviación estándar (KB)': np.std(diferencias, ddof=1) if diferencias.size > 1 else np.nan,
        'máximo (KB)': np.max(diferencias) if diferencias.size > 0 else np.nan,
        'mínimo (KB)': np.min(diferencias) if diferencias.size > 0 else np.nan
    }

# Convertir a DataFrame
df_estadisticas_k_means = pd.DataFrame.from_dict(estadisticas_k_means, orient='index')

# Mostrar el DataFrame
df_estadisticas_k_means


,media (KB),mediana (KB),desviación estándar (KB),máximo (KB),mínimo (KB)
103690.82,219.126552,150.8440,294.922849,1492.116,0.024
1472755.90,255.832300,234.5830,187.513922,769.342,0.397
1618022.90,269.698346,256.9790,248.337372,992.659,0.511
1647413.14,226.280960,132.4300,264.560698,1022.450,0.053
1869241.20,203.724824,135.1030,221.029368,807.884,0.111
1914872.23,228.404304,204.8660,184.268063,764.345,0.299
2038116.21,251.543485,198.5590,243.324128,1139.397,0.317
211165.20,121.540117,89.3425,134.271960,707.225,0.085
2490939.10,182.633368,186.4115,155.514424,528.114,0.024
2572090.70,252.618742,150.8910,274.492118,900.512,0.093


### umbrales

In [58]:
# Crear un nuevo diccionario para almacenar las estadísticas
estadisticas_umbrales = {}

# Calcular estadísticas para cada genoma
for id_genoma, diferencias in resultados_diferencias_umbrales.items():
    diferencias = np.array(diferencias)  # Asegurar que sea un array de NumPy
    estadisticas_umbrales[id_genoma] = {
        'media (KB)': np.mean(diferencias) if diferencias.size > 0 else np.nan,
        'mediana (KB)': np.median(diferencias) if diferencias.size > 0 else np.nan,
        'desviación estándar (KB)': np.std(diferencias, ddof=1) if diferencias.size > 1 else np.nan,
        'máximo (KB)': np.max(diferencias) if diferencias.size > 0 else np.nan,
        'mínimo (KB)': np.min(diferencias) if diferencias.size > 0 else np.nan
    }

# Convertir a DataFrame
df_estadisticas_umbrales = pd.DataFrame.from_dict(estadisticas_umbrales, orient='index')

# Mostrar el Data
df_estadisticas_umbrales

,media (KB),mediana (KB),desviación estándar (KB),máximo (KB),mínimo (KB)
103690.82,131.582104,88.4120,126.593838,504.895,0.024
1472755.90,152.722000,122.5560,132.634734,486.946,0.166
1618022.90,166.207714,97.6785,192.237499,889.946,0.024
1647413.14,217.489038,190.2885,198.588659,894.304,26.995
1869241.20,135.111765,79.0660,157.186253,713.185,0.111
1914872.23,168.840516,158.5720,153.060586,575.445,0.202
2038116.21,162.091784,116.5000,145.713915,619.268,0.096
211165.20,100.894556,70.0085,98.825340,455.312,0.042
2490939.10,118.988052,84.7315,120.114058,494.130,0.024
2572090.70,149.826846,107.8905,150.035525,676.841,0.093


## Estadisticas globales

### k-means

In [49]:
# Concatenar todas las diferencias en una sola lista
todas_diferencias_k_means = np.concatenate(list(resultados_diferencias_k_means.values()))

# Calcular estadísticas globales
estadisticas_globales_k_means = {
    'media (KB)': np.mean(todas_diferencias_k_means),
    'mediana (KB)': np.median(todas_diferencias_k_means),
    'desviación estándar (KB)': np.std(todas_diferencias_k_means),
    'máximo (KB)': np.max(todas_diferencias_k_means),
    'mínimo (KB)': np.min(todas_diferencias_k_means)
}

# Mostrar estadísticas globales
print('Estadísticas globales:')
for key, value in estadisticas_globales_k_means.items():
    print(f'  {key}: {value:.2f}')


Estadísticas globales:
  media (KB): 213.57
  mediana (KB): 141.39
  desviación estándar (KB): 250.99
  máximo (KB): 2023.08
  mínimo (KB): 0.02


### umbrales

In [50]:
# Concatenar todas las diferencias en una sola lista
todas_diferencias_umbrales = np.concatenate(list(resultados_diferencias_umbrales.values()))

# Calcular estadísticas globales
estadisticas_globales_umbrales = {
    'media (KB)': np.mean(todas_diferencias_umbrales),
    'mediana (KB)': np.median(todas_diferencias_umbrales),
    'desviación estándar (KB)': np.std(todas_diferencias_umbrales),
    'máximo (KB)': np.max(todas_diferencias_umbrales),
    'mínimo (KB)': np.min(todas_diferencias_umbrales)
}

# Mostrar estadísticas globales
print('Estadísticas globales:')
for key, value in estadisticas_globales_umbrales.items():
    print(f'  {key}: {value:.2f}')

Estadísticas globales:
  media (KB): 147.42
  mediana (KB): 94.65
  desviación estándar (KB): 159.22
  máximo (KB): 1312.57
  mínimo (KB): 0.02


## Exportar los datos

### k-means

In [53]:
## Guardar el df *genes_copia* en un archivo .csv para poder importarlo para el siguiente notebook
# Guardar como pkl
import pickle

# Guardar el diccionario en un archivo
with open('resultados_en_pb_k_means.pkl', 'wb') as file:
    pickle.dump(resultados_diferencias_k_means, file)


In [55]:
### umbrales

In [54]:
## Guardar el df *genes_copia* en un archivo .csv para poder importarlo para el siguiente notebook
# Guardar como pkl
import pickle

# Guardar el diccionario en un archivo
with open('resultados_en_pb_umbrales.pkl', 'wb') as file:
    pickle.dump(resultados_diferencias_umbrales, file)


In [56]:
%autosave 30

Autosaving every 30 seconds
